In [1]:
import os
import pandas as pd
import pandas as pd
import json
from ydata_profiling import ProfileReport
from ydata_profiling.utils.cache import cache_file
import seaborn as sns

In [32]:
#load metadata
with open('data/processing/metadata.json', 'r') as f:
    definitions = json.load(f)

with open('data/processing/industry-recode.json', 'r') as f:
    industry_recode = json.load(f)

with open('data/processing/occupasion-recode.json', 'r') as f:
    occupasion_recode = json.load(f)

with open('data/processing/education-recoder.json', 'r') as f:
    education_recoder = json.load(f)

with open('data/processing/martial-recode.json', 'r') as f:
    martial_recoder = json.load(f)

with open('data/processing/unemployment.json', 'r') as f:
    unemployment_recoder = json.load(f)


In [33]:
#load the data
df = pd.read_csv('data/raw/census_income_learn.csv', header=None, names=definitions.keys())

In [36]:
# cleaning

# remove ignore col
try: 
    df.drop(columns=['ignore'], inplace=True)
except:
    pass

# fix year"
if df['year'].min() < 100:
    df['year'] = df['year'] + 1900

# map industry code
if df['detailed_industry_recode'].dtype == 'int64':
    df['detailed_industry_recode'] = df['detailed_industry_recode'].astype(str).map(industry_recode)

# map occupation code
if df['detailed_occupation_recode'].dtype == 'int64':
    df['detailed_occupation_recode'] = df['detailed_occupation_recode'].astype(str).map(occupasion_recode)

if 'Advanced Degree' not in df['education'].unique():
    df['education'] = df['education'].map(education_recoder)

if 'Married' not in df['marital_stat'].unique():
    df['marital_stat'] = df['marital_stat'].map(martial_recoder)

if ' Other job loser' in df['reason_for_unemployment'].unique():
    df['reason_for_unemployment'] = df['reason_for_unemployment'].map(unemployment_recoder)

# drop duplicates values
df.drop_duplicates(inplace=True)

In [34]:
unemployment_recoder

{' Not in universe': 'Not in universe',
 ' Job loser - on layoff': 'Job loser - layoff',
 ' Other job loser': 'Job loser - layoff',
 ' New entrant': 'New entrant',
 ' Re-entrant': 'Re-entrant',
 ' Job leaver': 'Job leaver'}

In [37]:
df['reason_for_unemployment'].value_counts()

reason_for_unemployment
Not in universe       190120
Job loser - layoff      3014
Re-entrant              2018
Job leaver               598
New entrant              437
Name: count, dtype: int64

In [24]:
df['reason_for_unemployment'].unique()

array([' Not in universe', ' Job loser - on layoff', ' Other job loser',
       ' New entrant', ' Re-entrant', ' Job leaver'], dtype=object)

In [12]:
df.shape

(196187, 42)

In [63]:
df['education_level'] = df['education'].map(education_recoder)
print(df['education_level'].value_counts())

Series([], Name: count, dtype: int64)


In [65]:
df['education'].replace(education_recoder, inplace=True)

,0,1,2,3,4
age,73,58,18,9,10
class_of_worker,Not in universe,Self-employed-not incorporated,Not in universe,Not in universe,Not in universe
detailed_industry_recode,Not in universe or children,Manufacturing-durable goods,Not in universe or children,Not in universe or children,Not in universe or children
detailed_occupation_recode,Not in universe,Automobile mechanics and repairers,Not in universe,Not in universe,Not in universe
education,High School Graduate,Some College,Below High School,Children,Children
wage_per_hour,0,0,0,0,0
enroll_in_edu_inst_last_wk,Not in universe,Not in universe,High school,Not in universe,Not in universe
marital_stat,Widowed,Divorced,Never Married,Never Married,Never Married
major_industry_code,Not in universe or children,Construction,Not in universe or children,Not in universe or children,Not in universe or children
major_occupation_code,Not in universe,Precision production craft & repair,Not in universe,Not in universe,Not in universe


In [35]:
ProfileReport(df, title = 'census data raw report').to_file('data/raw/census_data_raw_report.html')

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]